# 1. Parsing HTML (allow 8 mins)<a href="#1.-Parsing-HTML-(allow-8-mins)" class="anchor-link">¶</a>

**Task** You will scrape and process simple html page located here
`candidateEvalData/webpage.html`

**Output** A dataframe of 1 row and 7 columns where the columns are:

1.  The name of the artist (Peter Doig)
2.  The name of the painting (The Architect's Home in the Ravine)
3.  Price realised in GBP (11 282 500)
4.  Price realised in USD (6 370 908)
5.  Estimates in GBP (10 000 000 , 15 000 000)
6.  Estimate in USD (14 509 999 , 21 764999)
7.  The url of the image of the painting
8.  Saledate of the painting (2016-02-11)

In [1]:
# import modules
from lxml import html
import re
import pandas as pd
from datetime import datetime


webpage_file_path = "candidateEvalData/webpage.html"

html_tree = html.parse(webpage_file_path)

# parse artist name
artist_x = "substring-before(//h1[contains(@class, 'lotName')]/text(), ' (')"
artist = html_tree.xpath(artist_x)
print("Artist name: {}\n".format(artist))

# parse painting name
painting_name_x = "//h2[contains(@class, 'itemName')]/*/text()"
painting_name = html_tree.xpath(painting_name_x)[0]
print("Painting name: {}\n".format(painting_name))


rm_comp = re.compile("(gbp|usd) |\(|\)", flags=re.IGNORECASE)
clean_price = lambda price: rm_comp.sub("", price).replace(",", " ")
clean_estimates = lambda estimates: clean_price(estimates).replace(" - ", ", ")
price_x = "//{tag}[contains(@*, '{descr}')]/text()"

# parse price GBP
gbp_price_x = price_x.format(tag="span", descr="PriceRealizedPrimary")
gbp_price = html_tree.xpath(gbp_price_x)
if gbp_price:
    gbp_price = clean_price(gbp_price[0])
    print("Price realised in GBP: {}\n".format(gbp_price))

# parse price US
usd_price_x = price_x.format(tag="div", descr="PriceRealizedSecondary")
usd_price = html_tree.xpath(usd_price_x)
if usd_price:
    usd_price = clean_price(usd_price[0])
    print("Price realised in USD: {}\n".format(usd_price))

# parse price GBP est
estimates_gbp_x = price_x.format(tag="span", descr="PriceEstimatedPrimary")
estimates_gbp = html_tree.xpath(estimates_gbp_x)
if estimates_gbp:
    estimates_gbp = clean_estimates(estimates_gbp[0])
    print("Estimates in GBP: {}\n".format(estimates_gbp))

# parse price US est
estimates_usd_x = price_x.format(tag="span", descr="PriceEstimatedSecondary")
estimates_usd = html_tree.xpath(estimates_usd_x)
if estimates_usd:
    estimates_usd = clean_estimates(estimates_usd[0])
    print("Estimates in USD: {}\n".format(estimates_usd))

# parse image link
image_x = "//*[contains(@id,'imgCarouselMain')]//a/@data-carousel-lg"
image = html_tree.xpath(image_x)[0]
print("The url of the image of the painting: {}\n".format(image))

# parse sale date
sale_date_x = "//span[contains(@*, 'SaleDate')]/text()"
sale_date_str = html_tree.xpath(sale_date_x)
if sale_date_str:
    sale_date_str = sale_date_str[0].replace(", ", "")
    sale_date = datetime.strptime(sale_date_str, "%d %B %Y").date()
    print("Saledate of the painting: {}".format(sale_date))


Artist name: Peter Doig

Painting name: The Architect's Home in the Ravine 

Price realised in GBP: 11 282 500

Price realised in USD: 16 370 908

Estimates in GBP: 10 000 000, 15 000 000

Estimates in USD: 14 509 999, 21 764 999

The url of the image of the painting: http://www.christies.com/lotfinderimages/D59730/peter_doig_the_architects_home_in_the_ravine_d5973059g.jpg

Saledate of the painting: 2016-02-11


# 2. Regex (allow 12 mins)<a href="#2.-Regex-(allow-12-mins)" class="anchor-link">¶</a>

For each example below, write a regex to process the string in rawDim to
extract the height, width and the depth (as float64 integers).

**Bonus**: Is there a single regex for all 5 examples ?

In [2]:
import re
import pandas as pd

dim_df = pd.read_csv("candidateEvalData/dim_df_correct.csv")
raw_dims = dim_df.rawDim.tolist()

dimension_exp = (
    r"(?:([0-9.]+) x )?([0-9.]+)\s*(?:x|×|by)\s*([0-9.]+)\s*(?=(?:cm|in))"
)
dimension_comp = re.compile(dimension_exp, flags=re.IGNORECASE)

processed_dimensions = []
# ('rawDim', 'height', 'width', 'depth')
for raw_dim in raw_dims:
    raw_dim = raw_dim.replace(",", ".")
    dimensions = dimension_comp.findall(raw_dim)
    dimensions = dimensions[-1] if "Image:" in raw_dim else dimensions[0]
    dimensions = list(float(dim) for dim in dimensions if dim)
    if "by" in raw_dim:
        conversion_unit = 2.54
        dimensions = list(map(lambda inch: inch * conversion_unit, dimensions))
    row = (raw_dim, *dimensions)
    processed_dimensions.append(row)


columns = dim_df.columns.tolist()
pd.DataFrame(processed_dimensions, columns=columns)


,rawDim,height,width,depth
0,19×52cm,19.0,52.0,NaN
1,50 x 66.4 cm,50.0,66.4,NaN
2,168.9 x 274.3 x 3.8 cm (66 1/2 x 108 x 1 1/2 in.),168.9,274.3,3.8
3,Sheet: 16 1/4 × 12 1/4 in. (41.3 × 31.1 cm) Im...,35.6,25.1,NaN
4,5 by 5in,12.7,12.7,NaN


# 3. Web crawler (allow 30 mins)<a href="#3.-Web-crawler-(allow-30-mins)" class="anchor-link">¶</a>

For this task we will crawl a gallery's listing of works available for
sale.

The listings are on <https://www.bearspace.co.uk/purchase>

Each listing leads to a detail page. Eg.

Eg. <https://www.bearspace.co.uk/product-page/tommy-by-olly-fathers>

**Task** Scrape all artworks available for sale. Please use scrapy for
this task.

**Output** Return a dataframe consisting of the following information

| url                                                              | title                 | media                                          | height_cm | width_cm | price_gbp |
|------------------------------------------------------------------|-----------------------|------------------------------------------------|-----------|----------|-----------|
| <https://www.bearspace.co.uk/product-page/tommy-by-olly-fathers> | Tommy by Olly Fathers | Spraypaint, board, card, digital print collage | 100       | 70       | 194       |

Your dataframe will have shape `(n,6)` where n is the total number of
works available to purchase.


In [3]:
import re
import scrapy
from scrapy.loader import ItemLoader
from itemloaders.processors import Compose, TakeFirst


def clean_price(price):
    """Return cleared price"""
    if price is not None:
        # remove currency symbol and comma
        cleared_price = price.replace(",", "").lstrip("£")
        return cleared_price


class ArtItem(scrapy.Item):
    url = scrapy.Field()
    title = scrapy.Field()
    media = scrapy.Field()
    height_cm = scrapy.Field()
    width_cm = scrapy.Field()
    price_gbp = scrapy.Field(
        input_processor=Compose(TakeFirst(), clean_price), serializer=int
    )


class BearspaceSpider(scrapy.Spider):
    name = "bearspace"
    allowe_domains = ["bearspace.co.uk"]

    def start_requests(self):
        if hasattr(self, "product_url"):
            yield scrapy.Request(
                url=self.product_url, callback=self.parse_product_detail_page
            )
        elif hasattr(self, "listing_url"):
            yield scrapy.Request(
                url=self.listing_url, callback=self.parse_listing_page
            )
        else:
            msg = (
                "Spider accept listing_url or product_url argument."
                " Please provide spider argument."
            )
            self.logger.warning(msg)

    def parse_listing_page(self, response):
        """Yield Request for each product detail page and next page"""
        detail_page_x = "//section//a[contains(@data-hook, 'details-link')]"
        yield from response.follow_all(
            xpath=detail_page_x,
            callback=self.parse_product_detail_page,
        )

        next_page_x = (
            "//ul/li[last()]/a"
            "[contains(@data-hook, 'pagination-link')]/@href"
        )
        if next_page_url := response.xpath(next_page_x).get():
            # send the next page response to parse_listing_page function
            yield response.request.replace(url=next_page_url)

    def parse_product_detail_page(self, response):
        """Yield populated item"""
        il = ItemLoader(item=ArtItem(), response=response)
        # set default output_processor to TakeFirst
        il.default_output_processor = TakeFirst()

        il.add_value("url", response.url)
        title_x = "//h1[contains(@*, 'product-title')]/text()"
        il.add_xpath("title", title_x)

        height_cm, width_cm, dim_text_idx = self.parse_dimensions(response)
        il.add_value("height_cm", height_cm)
        il.add_value("width_cm", width_cm)

        media = self.parse_media(response, dim_text_idx)
        il.add_value("media", media)

        price_x = '//span[contains(@*, "primary-price")]/text()'
        il.add_xpath("price_gbp", price_x)

        yield il.load_item()

    @staticmethod
    def parse_media(response, dim_text_idx):
        """Return media text determined based on dimension text position"""
        desc_text_x = "//pre[contains(@*, 'desc')]//text()[normalize-space()]"
        desc_texts = response.xpath(desc_text_x).getall()
        if desc_texts and dim_text_idx is not None:
            if dim_text_idx == 0:
                return desc_texts[dim_text_idx + 1]
            return desc_texts[dim_text_idx - 1]
        return None

    @staticmethod
    def parse_dimensions(response):
        """Return height, width, and dimension text position"""
        desc_text_x = "//pre[contains(@*, 'desc')]//text()[normalize-space()]"
        desc_texts = response.xpath(desc_text_x).getall()
        dimension_exp = r"([0-9.]+)[A-z\s]*x[A-z\s]*([0-9.]+)"
        dimension_comp = re.compile(dimension_exp, flags=re.IGNORECASE)
        for idx, tag_text in enumerate(desc_texts):
            if "diam" in tag_text.lower():
                diam = re.findall(r"([0-9.]+)", tag_text)[0]
                return diam, diam, idx
            elif dimensions := dimension_comp.findall(tag_text):
                height_cm, width_cm = dimensions[0][:2]
                return height_cm, width_cm, idx

        return None, None, None


import time
import pandas as pd
import scrapy.signals
from scrapy.crawler import CrawlerProcess


settings = {
    "AUTOTHROTTLE_ENABLED": True,
    "AUTOTHROTTLE_START_DELAY": 0.3,
    "AUTOTHROTTLE_MAX_DELAY": 1,
    "AUTOTHROTTLE_TARGET_CONCURRENCY": 6,
    "ROBOTSTXT_OBEY": False,
    "LOG_FILE": "logs/{}.log".format(int(time.time())),
}

items = []


def append_processed_item(item):
    items.append(dict(item))


crawler_process = CrawlerProcess(settings=settings)
crawler = crawler_process.create_crawler(BearspaceSpider)
crawler.signals.connect(
    append_processed_item, signal=scrapy.signals.item_scraped
)

test_urls = [
    "https://www.bearspace.co.uk/product-page/then-all-was-calm",
    "https://www.bearspace.co.uk/product-page/tommy-by-olly-fathers",
    "https://www.bearspace.co.uk/product-page/portal-8-by-jane-ward",
]
spider_kwargs = {
    "listing_url": "https://www.bearspace.co.uk/purchase"
    # "product_url": test_urls[0]
}
# pass kwargs to Spider init
crawler_process.crawl(crawler_or_spidercls=crawler, **spider_kwargs)
crawler_process.start()


columns_seq = ("url", "title", "media", "height_cm", "width_cm", "price_gbp")
df = pd.DataFrame.from_records(items, columns=columns_seq)
print("Shape of result dataframe: {}\n".format(df.shape))
df.to_csv("crawler_results.csv", index=None)

df.head(10)

Shape of result dataframe: (199, 6)



,url,title,media,height_cm,width_cm,price_gbp
0,https://www.bearspace.co.uk/product-page/p10-b...,P10 by Olly Fathers,Acrylic paint on sprayed board,30,30,240.00
1,https://www.bearspace.co.uk/product-page/off-r...,OFF ROAD by Lucy Baker,"Oil paint, spray paint & gold leaf on board",30,40,340.00
2,https://www.bearspace.co.uk/product-page/the-h...,The Hurricane by Lucy Baker,"Oil paint, spray paint & gold leaf on board",110,130,1600.00
3,https://www.bearspace.co.uk/product-page/amazo...,Amazons by Lucy Baker,"Oil paint, spray paint & gold leaf on board",110,130,1600.00
4,https://www.bearspace.co.uk/product-page/candy...,Candy Flop by Olly Fathers,Wall based sculpture,127,53,3400.00
5,https://www.bearspace.co.uk/product-page/cross...,Crossroads by Olly Fathers,Acrylic paint on sprayed board,120,120,4000.00
6,https://www.bearspace.co.uk/product-page/overl...,Overland I by Jane Ward,"Archival Digital print on stretched canvas, fi...",90,120,1400.00
7,https://www.bearspace.co.uk/product-page/tommy...,Tommy by Olly Fathers,"Spraypaint, board, card, digital print collage",100,70,1940.00
8,https://www.bearspace.co.uk/product-page/const...,Constructed Veneer IV by Olly Fathers,"Various wood veneers, various woods, board",38,32,1100.00
9,https://www.bearspace.co.uk/product-page/const...,Constructed Veneer VII by Olly Fathers,"Various wood veneers, various woods, board",34,25,940.00


# 4. Data (allow 5 mins)<a href="#4.-Data-(allow-5-mins)" class="anchor-link">¶</a>

There is a `nycflights13` database with the following tables:

1.  flights connects to planes via a single variable, tailnum.
2.  flights connects to airlines through the carrier variable.
3.  flights connects to airports in two ways: via the origin and dest
    variables.
4.  flights connects to weather via origin (the location), and year,
    month, day, and hour (the time).

## Joins<a href="#Joins" class="anchor-link">¶</a>

**Task 1 :** Describe `inner join`, `left join`, `right join`,
`full join`.

**Task 2 :** Write the `SQL` to do the following:

1.  Add full airline name to the `flights` dataframe and show the
    arr_time, origin, dest and the name of the airline.

2.  Filter resulting data.frame to include only flights containing the
    word *JetBlue*

3.  Summarise the total number of flights by origin in ascending.

4.  Filter resulting data.frame to return only origins with more than
    100 flights.

Your final dataframe would look like this

| origin | numFlights |
|--------|------------|
| JFK    | 148        |

**Output:** SQL query (no need to execute)

See data below

In [4]:
import pandas as pd


flights = pd.read_csv("candidateEvalData/flights.csv")
airports = pd.read_csv("candidateEvalData/airports.csv")
weather = pd.read_csv("candidateEvalData/weather.csv")
airlines = pd.read_csv("candidateEvalData/airlines.csv")

flights.head(3)

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01 05:00:00
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01 05:00:00
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01 05:00:00


In [5]:
airports.head(3)

,faa,name,lat,lon,alt,tz,dst
0,04G,Lansdowne Airport,41.130472,-80.619583,1044,-5.0,A
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-5.0,A
2,06C,Schaumburg Regional,41.989341,-88.101243,801,-6.0,A


In [6]:
weather.head(3)

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013.0,1.0,1,0,37.04,21.92,53.97,230.0,10.35702,11.918651,0.0,1013.9,10.0,2013-01-01T00:00:00Z
1,EWR,2013.0,1.0,1,1,37.04,21.92,53.97,230.0,13.80936,15.891535,0.0,1013.0,10.0,2013-01-01T01:00:00Z
2,EWR,2013.0,1.0,1,2,37.94,21.92,52.09,230.0,12.65858,14.567241,0.0,1012.6,10.0,2013-01-01T02:00:00Z


In [7]:
airlines.head(3)

,carrier,name
0,9E,Endeavor Air Inc.
1,AA,American Airlines Inc.
2,AS,Alaska Airlines Inc.


**Task 1:** SQL concepts

Joins are helpful to combine data across tables column wise. The ON and WHERE condition component affects the query output. For example, table A is left and table B is the right join predicate. 

- Table A: weather

- Table B: flights 

**Inner join** returns matched records from tables. 

**Left join** returns all records from table A and matched records from table B. 

**Right join** returns matched and unmatched records from table B and matched records from table A

**Full join** combines all records from tables. Unmatched attribute are filled with NULL.

**Task 2:** SQL statements

1.  Add full airline name to the `flights` dataframe and show the
    arr_time, origin, dest and the name of the airline.
```
select 
arr_time, origin, dest as destination, name as airline_name 
from flights 
join airlines on flights.carrier=airlines.carrier
```

2.  Filter resulting data.frame to include only flights containing the
    word *JetBlue*
```
select 
arr_time, origin, dest as destination, name as airline_name 
from flights
join airlines on flights.carrier=airlines.carrier
where airlines.name like 'JetBlue%'
```

3.  Summarise the total number of flights by origin in ascending.
```
select 
origin, count(origin) as total_flights 
from flights 
group by origin order by total_flights asc
```

4.  Filter resulting data.frame to return only origins with more than
    100 flights.
```
select 
origin, count(origin) as numFlights 
from flights 
group by origin having numFlights > 100
order by numFlights asc
```